# Kas süntaks on konserveerunud, 10 000 juhuslikul sulgudega lauset

In [1]:
import csv
import pandas as pd
import json

Loeme laused failist.

In [2]:
sentencesTSV = 'result/1_sentences_with_brackets.tsv'

df = pd.read_csv(sentencesTSV, sep='\t', header=None, quoting=csv.QUOTE_NONE)  
df.shape

(955564, 3)

In [3]:
df.head()

,0,1,2
0,0,484,Üks tolle õhtu süüdlasi oli kitarrist Sven Lõh...
1,1,152,Modelli ja modelliagentuuri omaniku Beatrice (...
2,1,429,Kui seni on Beatrice oma suhet DJ Priit Kuusik...
3,1,2906,Beatricel on Jürgen ( Jürgen Kaljuvee - toim )...
4,2,151,Eesti Kontserdi direktorile ja Arsise kellade ...


Ajame read juhuslikku järjekorda

In [4]:
def shuffle_rows(df):
    return df.sample(frac=1).reset_index(drop=True)

df = shuffle_rows(df)
df.head()

,0,1,2
0,202554,48,Vene-Balti sadamasse on sõitnud Gruusia lippu ...
1,700923,48988,"Selline seadus ( Rahvastikuregistri seadus , p..."
2,374958,91934,"( paus ) Rahvas aga ütleb : ( paus ) "" Vaenela..."
3,427790,2580,"Väljapääs peitub talumajanduse ja -kultuuri , ..."
4,652005,0,Mehitamata Vene veolaev Progress M-60 põkkus e...


### 10 000 esimest juhuslikku lauset

Laseme süntaksi peale.
Eemaldame sulgudega eraldatud sõned.
Laseme lühendatud lausele uuesti süntaksi peale, kontrollime, kas süntaks on konserveerunud.

In [5]:
import stanza
import os
from estnltk import Text
from estnltk.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger
from estnltk.converters.conll_exporter import  sentence_to_conll
import numpy as np


In [6]:
model_path = ".../estnltk-version_1.6/estnltk/taggers/syntax/stanza_tagger/stanza_resources"
model_path = '/Users/rabauti/stanza-models/stanza_recources'
input_type="morph_extended"
#stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, resources_path=model_path)
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)


In [7]:
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
from collections import defaultdict
from textwrap import wrap
import networkx as nx

class graphFunctions:
    
    # tipu leidmine atribuudi väärtuse järgi
    def get_nodes_by_attributes(G,  attrname, attrvalue ):
        nodes = defaultdict(list)
        {nodes[v].append(k) for k, v in nx.get_node_attributes(G,attrname).items()}
        if attrvalue in nodes:
            return dict(nodes)[attrvalue]
        return []

    # graafi joonistamine 
    # tipp - lemma
    # serv - deprel
    
    #TODO params to **kwargs
    def drawGraph(G, title=None, filename=None, highlight=[]):
        
        # soovitud tipud punaseks
        color_map = ['red' if node in highlight else 'lightskyblue' for node in G]
        
        # joonise suurus, et enamik puudest ära mahuks
        plt.rcParams["figure.figsize"] = (18.5, 10.5)
        
        #pealkiri
        if title:
            title = ("\n".join(wrap( title, 120)))
            plt.title(title)
        
        pos = graphviz_layout(G, prog='dot')
        labels = nx.get_node_attributes(G, 'lemma')
        nx.draw(G, pos, cmap = plt.get_cmap('jet'),labels=labels, with_labels=True, node_color=color_map)
        edge_labels = nx.get_edge_attributes(G, 'deprel')
        nx.draw_networkx_edge_labels(G, pos, edge_labels)
        
        #kui failinimi, siis salvestame faili
        #kui pole, siis joonistame väljundisse
        if filename:
            plt.savefig(f'{filename}.png', dpi=100)
        else:
            plt.show()
        plt.clf()
    
    # conllu lause objektist graafi tegemine
    def make_graph_conllu(sentence):
        G = nx.DiGraph()
        for data in sentence:
            if isinstance(data['id'], int):
                #paneme graafi kokku
                G.add_node(data['id'], id=data['id'], lemma=data['lemma'], pos=data['upostag'], deprel=data['deprel'], form=data['form'])
                G.add_edge(data['id'] - data['id'] + data['head'], data['id'], deprel = data['deprel'])
        return G
    
    # stanza stanza_syntax objektist graafi tegemine
    def make_graph_stanza(sentence):
        G = nx.DiGraph()
        for data in sentence:
            #print (data)
            if isinstance(data['id'], int):
                #paneme graafi kokku
                G.add_node(data['id'], id=data['id'], lemma=data['lemma'], pos=data['upostag'], deprel=data['deprel'], form=data.text)
                G.add_edge(data['id'] - data['id'] + data['head'], data['id'], deprel = data['deprel'])
        return G

    # lyhim tee graafi tippude vahel ning nn reversed kaartega graafist sama
    def get_shortest_paths(G):

        # lyhim tee tippude vahel
        path = nx.all_pairs_shortest_path_length(G)
        path_reversed = nx.all_pairs_shortest_path_length(G.reverse())
        # kauguste maatriksid
        dpath = {x[0]:x[1] for x in path}
        dpath_reversed = {x[0]:x[1] for x in path}
        return {'dict': path,  'dict_reversed': path_reversed, 'matrix': dpath, 'matrix_reversed': dpath_reversed}

    #tagastab array-na syntaksipuu graafi propreteid, tippu 0 ignoreerib
    def get_prop(graph, property_name):
        return [graph.nodes[node][property_name] for node in sorted([node for node in graph.nodes]) if node]
    
    #leiab, millised tipud suuremast graafist1 on puudu graafis2
    def get_nodes_diff(graph1, graph2):
        return [ node for node in graph1 if not node in graph2 ]

In [8]:
        
def analyze_as_graph(text, stanza_tagger):
    #uus analüüs
    short_sent_txt = Text(text)
    short_sent_txt.analyse('all')
    stanza_tagger.tag(short_sent_txt)
    new_graph = graphFunctions.make_graph_stanza(short_sent_txt.stanza_syntax)
    return new_graph

#add blanks at position of indexes instead of removed words
def add_blanks(a, indexes, blank='_'):
    array=a.copy()
    for ind in sorted(indexes):   
        array.insert(ind-1, blank)
       
    return array

def remove_removed(a, indexes):
    #print (array, indexes)
    array=a.copy()
    for ind in reversed(sorted(indexes)):
        array.pop(ind-1)
    return array


def is_equal(arr1, arr2):
    if not len(arr1) == len(arr2):
        return False
    for i in range(len(arr1)):
        if not arr1[i] == arr2[i]:
            return False
    return True

def remove_brackets(inputG):
    
    G = inputG.copy()
    # eeldame, et sulud ( ) on peale tokeniseerimist eraldi lemmaks märgendatud
    # läbime Graafi tipud sõnade järjekorras, korjame kokku sellised nodeId
    # mis algavad ( ja lõpevad ) 
    # eemaldame, ei arvesta süntaksipuu struktuuriga, korjame kokku ainult nodeid-d
    
    
    leftNodes1 = []
    leftNodes2 = sorted([n for n in G.nodes])
    nodes_to_remove = []
    while not len(leftNodes1) == len(leftNodes2):
        
        in_b = False
        leftNodes1 = leftNodes2.copy()
        remove = []
        for n in leftNodes2:
            #ignoreerime null tippu
            if not n: continue
            token = G.nodes[n]['form']
            if not in_b and token == '(':
                remove.append(n)
                in_b = True
                continue
            elif in_b  and  token == '(':
                remove=[]
                remove.append(n)
                in_b = True
                continue
            elif in_b and token == ')':
                #print ('siin break', n, G.nodes[n]['lemma'])
                remove.append(n)
                in_b = False
                break
            elif in_b:
                #print ('siin', n, G.nodes[n]['lemma'])
                remove.append(n)
        #kui lõpeatavad sulgu ei tulnud
        if in_b:
            remove = []
        for n in remove:
            leftNodes2.remove(n) 
            nodes_to_remove.append(n)
        
        #print ('leftNodes1', len(leftNodes1))
        #print ('leftNodes2', len(leftNodes2))
        
        #print ('remove', remove)
    
    for n in nodes_to_remove:
        G.remove_node(n)
    
    return G


In [9]:
%%time


#a) osal lausetest muutus tokeniseerimine, mida nendega teha
#b) ignoreerime laused, mis asuvad üleni sulgude vahel]

# deprel to remove
dprl = "brackets"

RESULTS_PATH = "./result/"


count = 10000
iterations = 2


stats = {'iteration': []
             , 'total_sentences': []
             , 'syntax_changed': []
             , 'syntax_conserved': []
             , 'allchanges': []
        }

for i in range(iterations):    
    iteration = i+1
    df = shuffle_rows(df)
    print (f"Iteratsioon {iteration}")
    sentencesFilename = f"{RESULTS_PATH}2_sentences_{dprl}_set_{iteration}.tsv"
   
    sentencesF = open(sentencesFilename, 'w', encoding='utf-8')
   
    # create the csv writer
    wSentences = csv.writer(sentencesF, delimiter='\t')
  


    wSentences.writerow(('collectionID', 'sentenceStart', 'changed', 'changes', 'original', 'short',))
    
    proc_txt_no = 0 # counter for texts
    no_rem_dprlss = 0   # count number of removed deprels
    no_changed_syntaxs = 0   # count sentences where syntax changed

    sentences_count = 0
    syntax_conserved = 0
    syntax_changed = 0

    all_changes = {}

    for ind in range(df.shape[0]):
        #print (ind)
        
        proc_txt_no = df[0][ind]
        sent_span_start = df[1][ind]

        sentence = df[2][ind]
        
        
        
         
        #originaallause analüüs
        g_orig = analyze_as_graph(sentence, stanza_tagger)
        deprel_origin = graphFunctions.get_prop(g_orig, 'deprel')
        
        #kontrollime, et sulud on sees?
        # siia kontroll, et advmod ikka sees
        #if not dprl in sent.deprel: continue
        
        sentences_count += 1
        
        #graphFunctions.drawGraph(g_orig, sentence)
    
        g_short = remove_brackets(g_orig)
    
        #kui tekst muutus, siis:
        #teeme uue lauseanalüüsi Stanzaga
        #teisendame graafiks
        #graafist küsime deprelid
        #print (sentence)
        nodes_diff = graphFunctions.get_nodes_diff(g_orig, g_short)
        
        short_text = " ".join(graphFunctions.get_prop(g_short, 'form'))
        if 1 in nodes_diff and len(short_text):
            short_text = short_text[0].upper() + short_text[1:]
    
        
        #graphFunctions.drawGraph(g_short, short_text )
    
        #ignoreerime lauseid, mille tekst oli pleni sulgude vahel
        if short_text == sentence or short_text == '' :
            continue
        #jäi samaks
        
        #print ('here')
        g_new = analyze_as_graph(short_text, stanza_tagger)
        #graphFunctions.drawGraph(g_new, short_text)
    
    
        deprel_new = graphFunctions.get_prop(g_new, 'deprel')
        depreln_new_with_blanks = add_blanks(deprel_new,nodes_diff)
        
        #if len(deprel_new) > 
        
        original_removed = remove_removed(deprel_origin, nodes_diff)
        
        conserved = is_equal(deprel_new, original_removed)
        if not conserved:
            syntax_changed +=1
            pass
            #print ()
            #print (sentence)
            #print (short_text)
            
            #print (" ".join(deprel_origin))
            #print (" ".join(depreln_new_with_blanks))
            
            #print ()
            #print (" ".join(original_removed))
            #print (" ".join(deprel_new))
            
            #graphFunctions.drawGraph(g_orig, sentence)
            #graphFunctions.drawGraph(g_new, short_text)
        else:
            syntax_conserved +=1
            
            
        
        #all_changes = {k: all_changes.get(k, 0) + result['changes'].get(k, 0) for k in set(all_changes) | set(result['changes'])}
       
        wSentences.writerow( [str(proc_txt_no) \
                , str(sent_span_start) \
                , str(int(not conserved)) \
                , '' \
                , sentence \
                , short_text  ], )
        
        wSentences.writerow( [str(proc_txt_no) \
                , str(sent_span_start) \
                , str(int(not conserved)) \
                , '' \
                , deprel_origin \
                , depreln_new_with_blanks  ], )
       

        if sentences_count >= count: break 
        
    sentencesF.close()

    stats['iteration'].append(iteration)
    stats['syntax_changed'].append(syntax_changed)
    stats['syntax_conserved'].append(syntax_conserved)
    stats['total_sentences'].append(sentences_count)
    stats['allchanges'].append(json.dumps(all_changes))
    
    print (f'Teisendatud {sentences_count} lauset.')
    print (f'Kokku vaadati {ind+1} lauset.')
    print (f'Süntaks konserveerunud {syntax_conserved} lausel.')

    print (all_changes)


  

Iteratsioon 1
Teisendatud 10000 lauset.
Kokku vaadati 10000 lauset.
Süntaks konserveerunud 5518 lausel.
{}
Iteratsioon 2
Teisendatud 10000 lauset.
Kokku vaadati 10000 lauset.
Süntaks konserveerunud 5517 lausel.
{}
CPU times: user 43min 12s, sys: 2min 9s, total: 45min 22s
Wall time: 44min 58s


Leiame 2 korda statistika 10000 juhusliku lause kohta, milles esinesid sulud .
    * mitmes lauses süntaks konserveerub,
    * mitmes lauses süntaks ei konserveeru. 

Salvestame need läbivaadatud laused failidesse, et saaks vajadusel kontrollida.

In [10]:
dfStat = pd.DataFrame(stats)
dfStat.to_csv(f"{RESULTS_PATH}2_{dprl}_stats.tsv",sep='\t', index=False)

In [11]:
dfChanged = pd.read_csv(sentencesFilename, sep='\t', quotechar = '"')  
dfChanged.shape

(19260, 6)

In [12]:
dfChanged.sample(10)

,collectionID,sentenceStart,changed,changes,original,short
12764,158218,1390,1,NaN,"Teises paaris sai TÜ/Eeden nädalavahetusel kaks 3 : 2 ( 24 , -24 , 16 , -21 , 11 ja -20 , -17 , 15 , 23 , 13 ) võitu Viljandi Metallitööstus/Volle üle 3 : 2 ( 24 , -24 , 16 , -21 , 11 ) ja mängib poolfinaalis põhiturniiri võitja ESS Pärnuga .",Teises paaris sai TÜ / Eeden nädalavahetusel kaks 3 : 2 võitu Viljandi Metallitööstus / Volle üle 3 : 2 ja mängib poolfinaalis põhiturniiri võitja ESS Pärnuga .
1434,402486,5806,0,NaN,"Üldjuhul ei ole mõistlik rakendada mõnes majandusharus kaitsetolle ja subsiidiume , millega paratamatult moonutataks majanduse konkurentsitingimusi ja genereeritaks laiskust ja võimetust ( nagu on näidanud Läti ja Leedu kogemused ) .","Üldjuhul ei ole mõistlik rakendada mõnes majandusharus kaitsetolle ja subsiidiume , millega paratamatult moonutataks majanduse konkurentsitingimusi ja genereeritaks laiskust ja võimetust ."
321,670395,7050,1,NaN,"['parataxis', 'root', 'flat', 'flat', 'flat', 'flat', 'flat', 'flat', 'punct', 'parataxis', 'punct', 'flat', 'nummod', 'punct', 'flat', 'punct', 'parataxis', 'flat', 'flat', 'nummod', 'punct', 'conj', 'punct', 'nummod', 'punct', 'punct', 'conj', 'flat', 'flat', 'flat', 'punct', 'parataxis', 'punct', 'nmod', 'nummod', 'punct', 'flat', 'punct', 'parataxis', 'flat', 'flat', 'nummod', 'punct', 'conj', 'punct', 'nummod', 'punct', 'punct']","['parataxis', 'root', 'flat', 'flat', 'flat', 'flat', 'flat', 'flat', '_', '_', '_', 'flat', 'flat', 'punct', 'flat', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'punct', 'conj', 'appos', 'flat', 'flat', '_', '_', '_', 'flat', 'nummod', 'punct', 'conj', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']"
7103,665759,32104,1,NaN,"['punct', 'parataxis', 'punct', 'mark', 'det', 'obj', 'advcl', 'det', 'obl', 'cc', 'nummod', 'conj', 'punct', 'root', 'nsubj', 'csubj', 'det', 'nmod', 'obj', 'punct']","['_', '_', '_', 'mark', 'det', 'obj', 'advcl', 'nummod', 'obl', 'cc', 'det', 'conj', 'punct', 'root', 'nsubj', 'csubj', 'det', 'nmod', 'obj', 'punct']"
13583,363814,411267,1,NaN,"['nsubj', 'root', 'nmod', 'nsubj', 'aux', 'conj', 'xcomp', 'amod', 'obj', 'nmod', 'nmod', 'cc', 'amod', 'nmod', 'obl', 'conj', 'amod', 'nmod', 'obl', 'compound:prt', 'nmod', 'obj', 'punct', 'amod', 'parataxis', 'nummod', 'punct', 'amod', 'nmod', 'nummod', 'punct', 'punct', 'mark', 'advcl', 'nummod', 'nsubj', 'punct', 'appos', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct']","['nsubj', 'root', 'nmod', 'nsubj', 'aux', 'conj', 'xcomp', 'amod', 'obj', 'nmod', 'obl', 'cc', 'amod', 'nmod', 'obl', 'conj', 'amod', 'nmod', 'obl', 'compound:prt', 'nmod', 'obj', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'punct', 'mark', 'advcl', 'nummod', 'nsubj', 'punct', 'appos', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct', 'conj', 'punct']"
16390,638720,84,0,NaN,0302 36 tuun ( Thunnus maccoyii ) :,0302 36 tuun :
10050,335653,699,0,NaN,"Endine asepresident Mesa ( pildil ) on ajakirjanik , ajaloolane ning TV uudisteankur .","Endine asepresident Mesa on ajakirjanik , ajaloolane ning TV uudisteankur ."
1261,234893,2871,0,NaN,"['nmod', 'obl', 'aux', 'root', 'punct', 'mark', 'det', 'nsubj:cop', 'cop', 'ccomp', 'advmod', 'amod', 'cc', 'conj', 'obl', 'punct', 'advmod', 'fixed', 'nsubj', 'aux', 'advcl', 'obl', 'punct', 'parataxis', 'obl', 'obj', 'punct', 'mark', 'advcl', 'det', 'nsubj:cop', 'acl', 'cop', 'punct', 'punct']","['nmod', 'obl', 'aux', 'root', 'punct', 'mark', 'det', 'nsubj:cop', 'cop', 'ccomp', 'advmod', 'amod', 'cc', 'conj', 'obl', 'punct', 'advmod', 'fixed', 'nsubj', 'aux', 'advcl', 'obl', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'punct